In [5]:

import os
import clickhouse_connect
import pymysql

from datetime import datetime
import pandas as pd
import numpy as np


## Reading Users from database

In [6]:
query = """SELECT
  users.ID AS UserID,
  users.CreatedAt AS JoinDate,
  FirstOrder.FirstOrderAt AS ActivationDate,
  DATEDIFF(FirstOrder.FirstOrderAt, users.CreatedAt) AS ActivationPeriod
FROM
  users
  LEFT JOIN `vw_users_first_order` AS FirstOrder ON users.ID = FirstOrder.UserID
  WHERE users.CreatedAt >= timestamp '2024-01-01 00:00:00.000' AND FirstOrder.FirstOrderAt IS NOT NULL"""


In [7]:
## Making database connection

HOST = os.environ.get('MYSQL_HOST')
USER = os.environ.get('MYSQL_USER')
PASSWORD = os.environ.get('MYSQL_PASSWORD')
conn = pymysql.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    db='mydb',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)

## Reading from database
try:
    with conn.cursor() as cursor:
        cursor.execute(query)
        rows = cursor.fetchall()
        users = pd.DataFrame(rows)
finally:
    conn.close()
    
print('Done Reading the Database!')
print("Shape of users:", users.shape)
del rows


Done Reading the Database!
Shape of users: (3555, 4)


In [8]:
# users['UserID'].nunique()
# users.head()
# users['ActivationPeriod'].max()
# users[users['ActivationPeriod']<=224]['ActivationPeriod'].value_counts(normalize=True)
users = users[users['ActivationPeriod']<=30]
users.shape


(2881, 4)

## Creating Events DataFrame

In [9]:
# Defining dataframe for the final result

events = users[['UserID', 'JoinDate', 'ActivationDate']].copy()
events.shape


(2881, 3)

## Utility Function

In [10]:
# Function that makes a tuple of event, session, and time

def tuplize(column, event, t=0):
    if t == 0:
        tup = (event, column['Time'])
    else:
        tup = (event, column['Time'], column['Category'], column['Name'], column['Price'])
    return tup


## Event Tables

In [11]:

# ClickHouse Database Settings
HOST = os.environ.get("CLICKHOUSE_HOST")
PORT = 8123
USER = os.environ.get("CLICKHOUSE_USERNAME")
PASSWORD = os.environ.get("CLICKHOUSE_PASSWORD")
DATABASE = os.environ["CLICKHOUSE_DATABASE"]


In [12]:

tables1 = ['advertisement_clicked', 'application_opened', 'application_updated', 
           'collection_clicked', 'first_order', 'product_list_filtered', 'product_list_viewed', 
           'products_searched', 'promotion_clicked', 'tooltip_clicked']
tables2 = [ 'product_added_to_wishlist', 'product_clicked', 'product_image_download', 
            'product_shared', 'product_video_download', 'product_video_play']#, 'product_viewed']

events1 = ['AddClick', 'AppOpen', 'AppUpdate', 'CollectionClick', 'FirstOrder', 'ProductListFilter', 
           'ProductListView', 'ProductSearch', 'PromotionClick', 'TooltipClick']
events2 = ['ProductAddtoList', 'ProductClick', 'ProductImageDownload',
                 'ProductShare', 'ProductVideoDownload', 'ProductVideoPlay']#, 'ProductView']

tables = [tables1, tables2]
ev = [events1, events2]


In [13]:

events = pd.DataFrame(columns=['UserID', 'Session'])
print("Events Reset!")


Events Reset!


In [14]:
## Making database connection

conn = clickhouse_connect.get_client(host=HOST, port=PORT, username=USER, 
                                         password=PASSWORD, secure=False)

for i in range(len(tables)):
    for j in range(len(tables[i])):
        event = ev[i][j]
        table = tables[i][j]
        print('\n', i, j, event,'---', table)
        if i == 0:
            query = """SELECT user_id AS UserID, context_session_id AS Session, timestamp AS Time 
                    FROM {0} 
                    WHERE UserID IS NOT NULL AND notEmpty(UserID) = 1 AND UserID <> ''
                    AND Session IS NOT NULL;""".format(table)
            columns=['UserID', 'Session', 'Time']
        
        else:
            query = """SELECT user_id AS UserID, context_session_id AS Session, timestamp AS Time, 
                   category AS Category, name AS Name, price AS Price
                   FROM {0}
                   WHERE UserID IS NOT NULL AND notEmpty(UserID) = 1 AND UserID <> ''
                   AND Session IS NOT NULL;""".format(table)
            columns=['UserID', 'Session', 'Time', 'Category', 'Name', 'Price']
        
        result = conn.query(query)
        rows = result.result_rows
        df = pd.DataFrame(rows, columns=columns)

        # Cleaning the UserID column
        df = df[df['UserID'].str.isdecimal()]
        df['UserID'] = df['UserID'].astype('int32')
        df['Session'] = df['Session'].astype('int16')
        
        df = df[df['UserID'].isin(users['UserID'])]
        df = pd.merge(users, df, on='UserID')
        df = df[df['Time'] < df['ActivationDate']]
        df['Event'] = df.apply(tuplize, t=i, event=event, axis=1)

        events = pd.concat([events, df[['UserID', 'Session', 'Event']].copy()])
        print('events after concat', events.shape)
                   
conn.close()    
print("Done!")



 0 0 AddClick --- advertisement_clicked
events after concat (239, 3)

 0 1 AppOpen --- application_opened
events after concat (223837, 3)

 0 2 AppUpdate --- application_updated
events after concat (224494, 3)

 0 3 CollectionClick --- collection_clicked
events after concat (233364, 3)

 0 4 FirstOrder --- first_order
events after concat (233668, 3)

 0 5 ProductListFilter --- product_list_filtered
events after concat (335137, 3)

 0 6 ProductListView --- product_list_viewed
events after concat (728008, 3)

 0 7 ProductSearch --- products_searched
events after concat (818326, 3)

 0 8 PromotionClick --- promotion_clicked
events after concat (830774, 3)

 0 9 TooltipClick --- tooltip_clicked
events after concat (833573, 3)

 1 0 ProductAddtoList --- product_added_to_wishlist
events after concat (857040, 3)

 1 1 ProductClick --- product_clicked
events after concat (1058270, 3)

 1 2 ProductImageDownload --- product_image_download
events after concat (1102088, 3)

 1 3 ProductShare --- 

In [52]:
events['UserID'].nunique()

2826

In [53]:
events.tail()

,UserID,Session,Event
82040,396784,4689,"(ProductVideoPlay, 2024-08-19 09:41:08, الكترو..."
82041,396784,4689,"(ProductVideoPlay, 2024-08-19 09:44:19, اجهزة ..."
82042,396784,4689,"(ProductVideoPlay, 2024-08-19 09:44:37, اجهزة ..."
82043,396784,-26374,"(ProductVideoPlay, 2024-08-19 19:20:54, الكترو..."
82054,397305,-6602,"(ProductVideoPlay, 2024-08-19 19:18:43, اجهزة ..."


## Reading product_viwed separately because of its size

In [15]:

# query= """select toInt64(product_viewed.user_id) AS PID, users_be.CreatedAt, FirstOrder.FirstOrderAt,
#     context_session_id AS Session, timestamp AS Time, category AS Category, name AS Name, price AS Price, 
#     toInt64(users_be.ID) AS UID, toInt64(FirstOrder.ID) AS FID
#     FROM product_viewed 
#     JOIN users_be ON UID= PID
#     JOIN `user_first_orders` AS FirstOrder ON PID = FID
#     WHERE (users_be.CreatedAt >= toDateTime('2024-01-01')) 
#         AND Session IS NOT NULL
#         AND Time < FirstOrder.FirstOrderAt;"""

query= """with my as (select toInt64(product_viewed.user_id) AS UserID, context_session_id AS Session, timestamp AS Time, FirstOrder.FirstOrderAt

    FROM product_viewed 
    JOIN users_be ON toInt64(users_be.ID)= toInt64(product_viewed.user_id)
    JOIN `user_first_orders` AS FirstOrder ON toInt64(product_viewed.user_id) = toInt64(FirstOrder.ID)
    WHERE (users_be.CreatedAt >= toDateTime('2024-01-01')) 
        AND Session IS NOT NULL
        AND Time < FirstOrder.FirstOrderAt)
           
select distinct * from my;"""
    

In [16]:

# Define your ClickHouse connection parameters
host = os.environ["CLICKHOUSE_HOST"]
port = 8123  # Default ClickHouse port, change if necessary
username = os.environ["CLICKHOUSE_USERNAME"]
password = os.environ["CLICKHOUSE_PASSWORD"]
database = os.environ["CLICKHOUSE_DATABASE"]

## Making database connection
conn = clickhouse_connect.get_client(host=HOST, port=PORT, username=USER, 
                                         password=PASSWORD, secure=False)
event = 'ProductView'
# columns=['UserID', 'JoinDate', 'ActivationDate', 'Session', 'Time', 'Category', 'Name', 'Price', 'UID', 'FID']
columns=['UserID', 'Session', 'Time', 'ActivationDate']

result = conn.query(query)
rows = result.result_rows
df = pd.DataFrame(rows, columns=columns)

# Cleaning the UserID column
df['UserID'] = df['UserID'].astype('int32')
df['Session'] = df['Session'].astype('int16')

df = df[df['UserID'].isin(users['UserID'])]
df['Event'] = df.apply(tuplize, t=0, event=event, axis=1)

events = pd.concat([events, df[['UserID', 'Session', 'Event']].copy()])
print('events after concat', events.shape)
           
conn.close()
events.reset_index(inplace=True, drop=True)
del df
print("Done!")      


events after concat (3125526, 3)
Done!


## Pickling the events dataframe

In [17]:

events.to_pickle("events.pkl")
print('finish')


finish


In [18]:
# events = pd.read_pickle("events.pkl")
# print("Shape of events:", events.shape)

## Event per User per Session

In [19]:

group = events.groupby(['UserID', 'Session']).agg(lambda x: sorted(list(x), key=lambda tup: tup[1]))
group['Lens'] = group['Event'].apply(len)
group.sort_values(by=['Lens'], inplace=True)
group.reset_index(inplace=True)
group.shape


(50985, 4)

In [84]:
group.head()
# group.iloc[14000,2]

,UserID,Session,Event,Lens
0,394346,-19340,"[(AppOpen, 2024-08-15 07:55:33)]",1
1,331514,32121,"[(ProductView, 2024-04-29 10:59:08)]",1
2,332255,-31079,"[(ProductView, 2024-04-23 10:00:20)]",1
3,332255,-30807,"[(ProductView, 2024-06-11 17:44:20)]",1
4,329605,5476,"[(ProductView, 2024-06-09 21:01:03)]",1


In [20]:
def sequence(row):
    seq = [tup[0] for tup in row['Event']]
    return seq

In [21]:
group['Sequence'] = group.apply(sequence, axis=1)

In [22]:
from itertools import groupby as gr

def deduplicate(row):
    return [key for key, _group in gr(row['Sequence'])]

group['CleanSequence'] = group.apply(deduplicate, axis=1)
group['lenClean'] = group['CleanSequence'].apply(len)
group = group[(group['lenClean']>4) & (group['lenClean'] < 20)]

# group['lenClean'].value_counts(normalize=True)


In [23]:
group.shape


(11949, 7)

In [24]:
# group[group['lenClean']<100]['lenClean'].hist()
vs = pd.DataFrame(group[group['lenClean']<50]['lenClean'].value_counts(normalize=True)).reset_index()
# vs.sort_values(by=['lenClean'], inplace=True)
# vs.columns


In [25]:

divent = {'AddClick': 'A', 'AppOpen': 'B', 'AppUpdate': 'C', 'CollectionClick': 'D', 'FirstOrder': 'E', 
          'ProductAddtoList': 'F', 'ProductClick': 'G', 'ProductImageDownload': 'H', 
          'ProductListFilter': 'I', 'ProductListView': 'J', 'ProductShare': 'K', 
          'ProductSearch': 'L', 'ProductVideoDownload': 'M', 'ProductVideoPlay': 'N', 
          'ProductView': 'O', 'PromotionClick': 'P', 'TooltipClick': 'Q'}
          

In [26]:

def alphabetize(column):
    replacer = divent.get
    vec = [replacer(n, n) for n in column['Sequence']]
    st = "".join(vec)
    return st

group['Vec'] = group.apply(alphabetize, axis=1)
group.head()

,UserID,Session,Event,Lens,Sequence,CleanSequence,lenClean,Vec
8939,376713,5129,"[(AppOpen, 2024-07-16 06:32:46), (ProductListV...",5,"[AppOpen, ProductListView, ProductView, Produc...","[AppOpen, ProductListView, ProductView, Produc...",5,BJOGH
8948,342268,-6673,"[(ProductView, 2024-05-02 19:13:55), (ProductL...",5,"[ProductView, ProductListView, ProductView, Pr...","[ProductView, ProductListView, ProductView, Pr...",5,OJOGN
8952,359427,-22507,"[(AppOpen, 2024-06-06 12:14:15), (ProductListV...",5,"[AppOpen, ProductListView, ProductClick, Produ...","[AppOpen, ProductListView, ProductClick, Produ...",5,BJGNM
8962,269098,-3931,"[(AppOpen, 2024-03-09 23:28:38), (ProductListV...",5,"[AppOpen, ProductListView, ProductView, Produc...","[AppOpen, ProductListView, ProductView, Produc...",5,BJOJB
8969,271211,24817,"[(AppOpen, 2024-02-21 20:47:06), (ProductListV...",5,"[AppOpen, ProductListView, ProductView, Produc...","[AppOpen, ProductListView, ProductView, Produc...",5,BJOJB


In [27]:
group.to_pickle("group.pkl")
print('finish')


finish


In [107]:

def levenshtein(a,b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a,b = b,a
        n,m = m,n
    current = range(n+1)
    for i in range(1,m+1):
        previous, current = current, [i]+[0]*n
        for j in range(1,n+1):
            add, delete = previous[j]+1, current[j-1]+1
            change = previous[j-1]
            if a[j-1] != b[i-1]:
                change = change + 1
            current[j] = min(add, delete, change) 
    return current[n]


In [ ]:
instanceKm = kmeans(X, initial_centers=initial_centers, metric=distance_metric(dist_measure))